In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sys
sys.path.append("../")
from component.message import cm

<IPython.core.display.Javascript object>

In [4]:
from component.model.recipe import Recipe
from component.tile.custom_aoi_tile import AoiTile
from component.tile.dashboard_tile import DashboardTile
from component.widget.alert_state import AlertState

ResizeTrigger()

VuetifyTemplate(template="\n        <style class='sepal-ui-script'>\n            .custom_map .jupyter-widgets.…

In [5]:
build_alert = AlertState()

In [6]:
build_alert

AlertState(class_='d-none', layout=None, old_class='mt-5', style_='margin: 0 !important;', text=True, type='in…

In [7]:
recipe = Recipe()
recipe.load_model()

In [8]:
aoi_tile = AoiTile()

In [9]:
dashboard_tile = DashboardTile()
dashboard_tile.build(recipe=recipe, build_alert=build_alert)

In [10]:
aoi_tile.build(recipe=recipe, build_alert=build_alert)

In [20]:
aoi_tile;

In [21]:
dashboard_tile;

### Test stats 

In [15]:
ee_features = recipe.seplan.aoi_model.get_ee_features()
ee_features

{'COL_Risaralda': {'ee_feature': <pygaul.AdmItems at 0x772c6ebe3250>,
  'color': '#b3842e'}}

In [16]:
from component.tile.dashboard_tile import DashboardTile
from component.widget.alert_state import AlertState

In [17]:
alert = AlertState()
alert

AlertState(class_='d-none', layout=None, old_class='mt-5', style_='margin: 0 !important;', text=True, type='in…

In [18]:
dashboard_tile = DashboardTile()

In [19]:
dashboard_tile.build(recipe, alert)

AttributeError: 'Recipe' object has no attribute 'dash_model'

In [14]:
dashboard_tile

DashboardTile(children=[DashToolBar(children=[Tooltip(children=['Save dashboard statistics as .csv'], close_de…

## Test statistics from recipe

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")
from component.scripts.statistics import parse_theme_stats, get_summary_statistics
from component.model.recipe import Recipe
from component.scripts.seplan import reduce_constraints
import ee
ee.Initialize()

<IPython.core.display.Javascript object>

ResizeTrigger()

In [3]:
# Load a previously created recipe
recipe = Recipe()
recipe.load_model()
recipe.load("/home/dguerrero/module_results/se.plan/recipes/test_cundinamarca.json")
recipe.seplan_aoi.aoi_model.set_object(); # we have to do this manually
aoi = recipe.seplan_aoi.aoi_model.feature_collection

In [8]:
summary_stats = get_summary_statistics(recipe.seplan)
summary_stats

[{'COL_Cundinamarca': {'benefit': [{'biodiversity_intactness': {'total': [0.88282625811616],
      'values': [0.8838698205429807]}},
    {'endangered_species': {'total': [15.02175306852444],
      'values': [15.846754387002877]}},
    {'ground_carbon': {'total': [81.91007175108024],
      'values': [81.8111435147813]}},
    {'forest_job': {'total': [0.011914094776501726],
      'values': [0.011997179747788371]}},
    {'woodfuel_harvest': {'total': [0.407807665975417],
      'values': [0.41001070769679226]}}],
   'color': '#b3842e',
   'constraint': [{'treecover_with_potential': {'total': [2444852.477004771],
      'values': [22.754402432843605]}},
    {'city_access': {'total': [2444852.4770066217],
      'values': [0.5431212158648957]}},
    {'ecozones': {'total': [2444852.4770057476],
      'values': [23.428423762423307]}},
    {'annual_rainfall': {'total': [2444852.477004152],
      'values': [62.37692296276993]}}],
   'cost': [{'opportunity_cost': {'total': [6705.203742898495],
    

In [9]:
parse_theme_stats(summary_stats)

{'benefit': {'biodiversity_intactness': {'values': [0.8838698205429807],
   'total': 0},
  'endangered_species': {'values': [15.846754387002877], 'total': 0},
  'ground_carbon': {'values': [81.8111435147813], 'total': 0},
  'forest_job': {'values': [0.011997179747788371], 'total': 0},
  'woodfuel_harvest': {'values': [0.41001070769679226], 'total': 0}},
 'constraint': {'treecover_with_potential': {'values': [22.754402432843605],
   'total': 0},
  'city_access': {'values': [0.5431212158648957], 'total': 0},
  'ecozones': {'values': [23.428423762423307], 'total': 0},
  'annual_rainfall': {'values': [62.37692296276993], 'total': 0}},
 'cost': {'opportunity_cost': {'values': [1914.5781778540886], 'total': 0},
  'implementation_cost': {'values': [464.1883588847593], 'total': 0}}}

## Calculate the image percent cover pixel area

In [ ]:
from component.scripts.statistics import get_image_percent_cover_pixelarea
from component.scripts.seplan import mask_image

In [ ]:
aoi = recipe.seplan.aoi_model.feature_collection

In [ ]:
asset_id = recipe.seplan.constraint_model.assets[0]
data_type = recipe.seplan.constraint_model.data_type[0]
maskout_values = recipe.seplan.constraint_model.values[0]

asset_id, data_type, maskout_values

In [ ]:
# maskout any image from the model

masked_image = mask_image(asset_id, data_type, maskout_values)

In [ ]:
get_image_percent_cover_pixelarea(masked_image, aoi, "perro").getInfo()

In [ ]:
from sepal_ui.mapping import SepalMap
import ee

ee.Initialize()

In [ ]:
map_ = SepalMap()
map_.addLayer(masked_image.unmask(0).randomVisualizer().clip(aoi))
map_

## Calculate the image sum on COSTS

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from component.scripts.statistics import get_image_sum
from component.scripts.seplan import reduce_constraints, asset_to_image

In [ ]:
aoi = recipe.seplan.aoi_model.feature_collection

In [ ]:
asset_id = recipe.seplan.cost_model.assets[0]
constraint_list = recipe.seplan.get_masked_constraints_list()
mask_out_areas = reduce_constraints(constraint_list)

In [ ]:
image = asset_to_image(asset_id)

In [ ]:
sum_img = image.updateMask(mask_out_areas).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=aoi,
    scale=100,
    maxPixels=1e13,
)
sum_img.values().get(0).getInfo()

In [ ]:
get_image_sum(image, aoi, mask_out_areas, "name").getInfo()

In [ ]:
area_ha = (
    ee.Image.pixelArea()
    .divide(10000)
    .reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=100,
        maxPixels=1e13,
    )
    .getInfo()
)

In [ ]:
map_.addLayer(constraint_list[0][0].clip(aoi), {}, "mask")

In [ ]:
map_ = SepalMap()
map_.addLayer(mask_out_areas.randomVisualizer().clip(aoi))
map_